<a href="https://colab.research.google.com/github/Suhail-46/Asteroid-Tracker-for-Earth/blob/main/NASA_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mysql-connector-python # installation of SQL connector

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 54.1 MB/s eta 0:00:00


In [ ]:
# Code used to scrape 10000 data from NASA API
import requests
import pprint
api_key = "zVrrXmXMHFlLP3ow0eNmYfJ2RWaMrjyKJi9eBYvl"
asteroids_data = [] # Created empty list to store the collection of values from scraping
target = 10000
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&api_key={api_key}" # Request given to collect data within date range of 2024-01-01 to 2024-01-07
response = requests.get(url)
data = response.json()
details = data["near_earth_objects"]
while len(asteroids_data) < target: # Loop used to collect 10000 data
  for date, ast_details in details.items(): # Loop used to collect the data based on range of date
      for ast in ast_details: # Loop used to extract the field
        asteroids_data.append(dict(
            id = ast["id"],
            neo_reference_id = ast["neo_reference_id"],
            name = ast["name"],
            absolute_magnitude_h = ast["absolute_magnitude_h"],
            estimated_diameter_min_km = ast["estimated_diameter"]["kilometers"]["estimated_diameter_min"],
            estimated_diameter_max_km = ast["estimated_diameter"]["kilometers"]["estimated_diameter_max"],
            is_potentially_hazardous_asteroid = ast["is_potentially_hazardous_asteroid"],
            close_approach_date = ast["close_approach_data"][0]["close_approach_date"],
            relative_velocity_kmph = ast["close_approach_data"][0]["relative_velocity"]["kilometers_per_hour"],
            astronomical = ast["close_approach_data"][0]["miss_distance"]["astronomical"],
            miss_distance_km = ast["close_approach_data"][0]["miss_distance"]["kilometers"],
            miss_distance_lunar = ast["close_approach_data"][0]["miss_distance"]["lunar"],
            orbiting_body = ast["close_approach_data"][0]["orbiting_body"]
        ))
        if len(asteroids_data) == target:
            break # Loop breaks once all fields of data collected
      if len(asteroids_data) == target:
        break # Loop breaks once all data collected within the range of date
url = data["links"]["next"] # The loops were applied to the next page of API
# Two list of data is extracted to feed on the table as per the requirement
asteroids_list = [] # Created an empty list to store data of asteroids with respect to the requirement of asteroids table
for asteroids in asteroids_data:
    asteroids_list.append((
        asteroids['id'],
        asteroids['name'],
        asteroids['absolute_magnitude_h'],
        asteroids['estimated_diameter_min_km'],
        asteroids['estimated_diameter_max_km'],
        asteroids['is_potentially_hazardous_asteroid']
    ))
close_approach_list = [] # Created an empty list to store data of close approach with respect to the requirement of close_approach table
for close_approach in asteroids_data:
    close_approach_list.append((
        close_approach['neo_reference_id'],
        close_approach['close_approach_date'],
        close_approach['relative_velocity_kmph'],
        close_approach['astronomical'],
        close_approach['miss_distance_km'],
        close_approach['miss_distance_lunar'],
        close_approach['orbiting_body']
    ))

In [ ]:
# Code to create table and insert the bulk of data into the created table
import mysql.connector
suhail = mysql.connector.connect(host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
                                 user="2XT5EKXhUVwb2JW.root",
                                 password="YO7YdnLJV098Hccw",
                                 port = 4000,
                                 database="NASA_Project") # Establish DB connection using TiDB cloud
connection = suhail.cursor()
connection.execute("create table asteroids (id int, name varchar(20), absolute_magnitude_h float, estimated_diameter_min_km float, estimated_diameter_max_km float, is_potentially_hazardous_asteroid boolean, constraint pk_id primary key (id))")
connection.execute("create table close_approach (neo_reference_id int, close_approach_date date, relative_velocity_kmph float, astronomical float, miss_distance_km float, miss_distance_lunar float, orbiting_body varchar(20), constraint fk_neo_reference_id foreign key (neo_reference_id) references asteroids(id))")
insert_query_1 = '''INSERT IGNORE INTO asteroids (id, name, absolute_magnitude_h, estimated_diameter_min_km, estimated_diameter_max_km, is_potentially_hazardous_asteroid) VALUES (%s, %s, %s, %s, %s, %s)'''
insert_query_2 = '''INSERT INTO close_approach (neo_reference_id, close_approach_date, relative_velocity_kmph, astronomical, miss_distance_km, miss_distance_lunar, orbiting_body) VALUES (%s, %s, %s, %s, %s, %s, %s)'''
connection.executemany(insert_query_1, asteroids_list)
connection.executemany(insert_query_2, close_approach_list)
suhail.commit()
print("Inserted asteroids and close_approach data into MySQL!")

Inserted asteroids and close_approach data into MySQL!


In [ ]:
!pip install streamlit streamlit_option_menu # installation of streamlit for data visualization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
# Writing code in .py file format used to support streamlit
%%writefile app.py
import streamlit as st
import pandas as pd
import mysql.connector
import datetime
suhail = mysql.connector.connect(host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
                                 user="2XT5EKXhUVwb2JW.root",
                                 password="YO7YdnLJV098Hccw",
                                 port = 4000,
                                 database="NASA_Project")
connection = suhail.cursor()
st.sidebar.title("🌠 Asteroid Approach")
st.sidebar.divider()
menu = st.sidebar.radio("Navigate",("Filter Criteria", "Queries"))
st.title("🚀 :blue[Asteroid Tracker for Earth] ☄️")
st.divider()
if menu == "Filter Criteria":
  left, middle, right = st.columns(3) # Organizing the data using sliders and select boxes
  min_mag = left.slider("Min Magnitude", 13.00, 32.61, (13.00, 32.61))
  velocity = middle.slider("Relative_velocity_kmph Range", 1418.21, 173071.83, (1418.21, 173071.83))
  min_dia = left.slider("Min Estimated Diameter(km)", 0.00, 4.62, (0.00, 4.62))
  max_dia = left.slider("Max Estimated Diameter(km)", 0.00, 10.33, (0.00, 10.33))
  astronomical = middle.slider("Astronomical unit", 0.00, 0.50, (0.00, 0.50))
  hazard = middle.selectbox("Only Show Potentially Hazardous", ("1", "0"))
  str_dt = right.date_input("Start Date", datetime.date(2024, 1, 1))
  end_dt = right.date_input("End Date", datetime.date(2024, 1, 7))
  button = st.button("Filter")
  if button:
    st.header("Filtered Asteroids", divider="blue")
    query = """select distinct a.id, a.name, a.absolute_magnitude_h, a.estimated_diameter_min_km, a.estimated_diameter_max_km,
              a.is_potentially_hazardous_asteroid, c.neo_reference_id, c.close_approach_date, c.relative_velocity_kmph, c.astronomical,
              c.miss_distance_km, c.miss_distance_lunar, c.orbiting_body
              from asteroids a
              join close_approach c on a.id = c.neo_reference_id
              where a.absolute_magnitude_h between %s and %s
              and a.estimated_diameter_min_km between %s and %s
              and a.estimated_diameter_max_km between %s and %s
              and c.relative_velocity_kmph between %s and %s
              and c.astronomical between %s and %s
              and c.close_approach_date between %s and %s
              and a.is_potentially_hazardous_asteroid = %s order by a.id""" # Query used to display data based on the applied filters over sliders and select box
    params = (min_mag[0], min_mag[1],
             min_dia[0], min_dia[1],
             max_dia[0], max_dia[1],
             velocity[0], velocity[1],
             astronomical[0], astronomical[1],
               str_dt, end_dt, int(hazard)) # Parameters are used to pass on based on the appiled filters
    connection.execute(query, params)
    records = connection.fetchall()
    df = pd.DataFrame(records, columns=[col[0] for col in connection.description])
    st.dataframe(df)
elif menu == "Queries": # Code written for another tab of side menu to diplay the quries based on the requirements
    option = st.selectbox(
        "Select your Query",
        (
            "Count of how many times each asteroid has approached Earth",
            "Average velocity of each asteroid over multiple approaches",
            "List top 10 fastest asteroids",
            "Potentially hazardous asteroids that have approached Earth more than 3 times",
            "Month with the most asteroid approaches",
            "Asteroid with the fastest ever approach speed",
            "Sorted Asteroids in descending order by maximum estimated diameter",
            "Asteroid whose closest approach is getting nearer over time",
            "Asteroids of its closest approach to Earth along with the date and miss distance",
            "Asteroids that approached Earth with velocity > 50,000 km/h",
            "Count of asteroids approaches happened per month",
            "Asteroid with the highest brightness",
            "Number of hazardous vs non-hazardous asteroids",
            "Asteroids that passed closer than the Moon (lesser than 1 LD), along with their close approach date and distance",
            "Asteroids that came within 0.05 AU"
        )
    )

    if option == "Count of how many times each asteroid has approached Earth":
        query1 = """select a.name, count(c.neo_reference_id) as count from asteroids a join close_approach c on a.id = c.neo_reference_id group by a.name"""
        df1 = pd.read_sql(query1, suhail)
        st.dataframe(df1)
    elif option == "Average velocity of each asteroid over multiple approaches":
        query2 = """select a.name, avg(c.relative_velocity_kmph) as average_velocity from asteroids a join close_approach c on a.id = c.neo_reference_id group by a.name"""
        df2 = pd.read_sql(query2, suhail)
        st.dataframe(df2)
    elif option == "List top 10 fastest asteroids":
        query3 = """select a.name, max(c.relative_velocity_kmph) as maximum_velocity from asteroids a join close_approach c on a.id = c.neo_reference_id group by a.name order by maximum_velocity desc limit 10"""
        df3 = pd.read_sql(query3, suhail)
        st.dataframe(df3)
    elif option == "Potentially hazardous asteroids that have approached Earth more than 3 times":
        query4 = """select a.name, count(c.neo_reference_id) as Approached_Earth_more_than_3_times from asteroids a join close_approach c on a.id = c.neo_reference_id where a.is_potentially_hazardous_asteroid = 1 group by a.name having count(c.neo_reference_id) > 3"""
        df4 = pd.read_sql(query4, suhail)
        st.dataframe(df4)
    elif option == "Month with the most asteroid approaches":
        query5 = """select monthname(close_approach_date) as Month_with_the_most_asteroid_approaches, count(*) as approach_count from close_approach group by Month_with_the_most_asteroid_approaches"""
        df5 = pd.read_sql(query5, suhail)
        st.dataframe(df5)
    elif option == "Asteroid with the fastest ever approach speed":
        query6 = """select a.name, max(c.relative_velocity_kmph) as maximum_speed from asteroids a join close_approach c on a.id = c.neo_reference_id group by a.name order by maximum_speed desc limit 1;"""
        df6 = pd.read_sql(query6, suhail)
        st.dataframe(df6)
    elif option == "Sorted Asteroids in descending order by maximum estimated diameter":
        query7 = """select name, estimated_diameter_max_km from asteroids order by estimated_diameter_max_km desc"""
        df7 = pd.read_sql(query7, suhail)
    elif option == "Asteroid whose closest approach is getting nearer over time":
        query8 = """select a.name, c.miss_distance_km from asteroids a join close_approach c on a.id = c.neo_reference_id order by close_approach_date"""
        df8 = pd.read_sql(query8, suhail)
        st.dataframe(df8)
    elif option == "Asteroids of its closest approach to Earth along with the date and miss distance":
        query9 = """select a.name, c.close_approach_date, c.miss_distance_km from asteroids a join close_approach c on a.id = c.neo_reference_id"""
        df9 = pd.read_sql(query9, suhail)
        st.dataframe(df9)
    elif option == "Asteroids that approached Earth with velocity > 50,000 km/h":
        query10 = """select distinct a.name from asteroids a join close_approach c on a.id = c.neo_reference_id where c.relative_velocity_kmph > 50000"""
        df10 = pd.read_sql(query10, suhail)
        st.dataframe(df10)
    elif option == "Count of asteroids approaches happened per month":
        query11 = """select month(close_approach_date), count(*) as approach_count from close_approach group by month(close_approach_date)"""
        df11 = pd.read_sql(query11, suhail)
        st.dataframe(df11)
    elif option == "Asteroid with the highest brightness":
        query12 = """select name, absolute_magnitude_h from asteroids order by absolute_magnitude_h limit 1"""
        df12 = pd.read_sql(query12, suhail)
        st.dataframe(df12)
    elif option == "Number of hazardous vs non-hazardous asteroids":
        query13 = """select is_potentially_hazardous_asteroid as hazardous_status, count(*) as asteroid_count from asteroids group by is_potentially_hazardous_asteroid"""
        df13 = pd.read_sql(query13, suhail)
        st.dataframe(df13)
    elif option == "Asteroids that passed closer than the Moon (lesser than 1 LD), along with their close approach date and distance":
        query14 = """select a.name, c.close_approach_date, c.miss_distance_lunar from asteroids a join close_approach c on a.id = c.neo_reference_id where c.miss_distance_lunar < 1 order by c.miss_distance_lunar"""
        df14 = pd.read_sql(query14, suhail)
        st.dataframe(df14)
    elif option == "Asteroids that came within 0.05 AU":
        query15 = """select a.name, c.astronomical from asteroids a join close_approach c on a.id = c.neo_reference_id where c.astronomical < 0.05"""
        df15 = pd.read_sql(query15, suhail)
        st.dataframe(df15)

Overwriting app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.81.185.30


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.81.185.30:8501

⠼⠴⠦⠧⠇⠏⠋your url is: https://tiny-pugs-tan.loca.lt
/content/app.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, suhail)
/content/app.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, suhail)
/content/app.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, suhail)
/conten